In [1]:
import os, sys, django, pandas as pd , numpy as np, requests, json, re, itertools, ftfy
from django.db.models import Q
from utils import TJData, setup_env, ElisAPI
# configura notebook para se utilizar recursos do django, se necessário
setup_env() 
from api import models as tj
# objeto para interagir com a minha API
api = ElisAPI('http://0.0.0.0:8000','elisdbadmin','elisdbpassword')


def retry_post(resource, r_post):
    r_post_old = r_post
    if r_post:
        r_post_retry = api.retry_post(resource,r_post)
        return r_post_old, r_post_retry
    return r_post_old

### Comarca
O bloco de código abaixo foi utilizado para analisar e decidir visualmente qual planilha referente a comarcas será utilizada, a planilha mapeada por `comarca` possui a coluna `COD_SERV` possui codigos distintos dos referenciados por demais planilhas
```
com_1 = TJData.read_csv(TJData.csv['comarca'],usecols=['COD_COM','NOME','NOME_REDU','COD_TJ'])
com_2 = TJData.read_csv(TJData.csv['comarca_2'], usecols=['COD_COMA','DESC_COMA','COD_TJ','DESC_REDU'])
com = pd.merge(left=com_1,right=com_2,how='inner',left_on='NOME_REDU', right_on='DESC_REDU')
```

#### Tratamento de dados

### Serventia

``` 
serv_1 = TJData.read_csv(TJData.csv['serventia_2'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'] )
serv_2 = TJData.read_csv(TJData.csv['serventias_full'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'])
serv = pd.merge(left=serv_1,right=serv_2,how='inner',left_on='COD_SERV', right_on='COD_SERV')
serv_outer = pd.merge(left=serv_1,right=serv_2,how='outer',left_on='COD_SERV', right_on='COD_SERV')
```
#### Tratamento de dados

### Competencias

### Tipo Personagem

### Assunto

### Classe

### Classe assunto

### Tipo movimento

### Tipo Andamento

### Cargo

### Funcionário

### Tipo Decisão Recurso

### Tipo Ato Juiz

### Ato Juiz

### Tipo Documento

### Processo

#### Obtem as serventias com função de Juizado Especial Cível das comarcas de Bangu e Campo Grande

In [2]:
serventias_jec = pd.DataFrame(
    list(map(lambda x: (x.comarca.desc_redu,x.comarca.desc_coma, x.cod_serv, x.desc_serv),
        tj.Serventia.objects.filter(
            Q(desc_serv__contains='Juizado Especial Cível'),
            Q(comarca__desc_redu='BAN') | Q(comarca__desc_redu='CGR')
    ))),
    columns=['comarca.desc_redu','comarca.desc_coma','cod_serv','desc_serv']
)
serventias_jec

,comarca.desc_redu,comarca.desc_coma,cod_serv,desc_serv
0,BAN,Regional de Bangu,204009,Cartório do 17º Juizado Especial Cível
1,BAN,Regional de Bangu,204509,17º Juizado Especial Cível
2,BAN,Regional de Bangu,204025,Cartório do 29º Juizado Especial Cível
3,BAN,Regional de Bangu,204525,29º Juizado Especial Cível
4,CGR,Regional de Campo Grande,205020,Cartório do 26º Juizado Especial Cível
5,CGR,Regional de Campo Grande,205520,26º Juizado Especial Cível
6,CGR,Regional de Campo Grande,205507,18º Juizado Especial Cível
7,CGR,Regional de Campo Grande,205007,Cartório do 18º Juizado Especial Cível


#### Obtem a lista de planilhas para processar resultantes codigos de serventias e do intervalo de anos selecinados 

### Andamento Processo (Sentença)

### Personagem

#### carrega dados codigo e nome do personagem especial (jurídica) e personagens pessoa fĩsicaa

#### carrega dados dos advogados

#### divide as associações aos processos entre personagens (fisico ou juridico) e advogados

### Documentos (Iniciais e Contestação)

### Arquivos similaridade

### zona de teste

In [3]:
from frontend import models, serializers

In [4]:
serializers.ComarcaServentiaDisponivelSerializer(models.ComarcaServentiaDisponivel.objects.all(),many=True)

ComarcaServentiaDisponivelSerializer(<QuerySet [<ComarcaServentiaDisponivel: Cartório do 17º Juizado Especial Cível - Regional de Bangu>, <ComarcaServentiaDisponivel: Cartório do 29º Juizado Especial Cível - Regional de Bangu>, <ComarcaServentiaDisponivel: Cartório do 18º Juizado Especial Cível - Regional de Campo Grande>, <ComarcaServentiaDisponivel: Cartório do 26º Juizado Especial Cível - Regional de Campo Grande>]>, many=True):
    id = IntegerField(label='ID', read_only=True)
    serventia = CharField(style={'base_template': 'textarea.html'})
    comarca = CharField(style={'base_template': 'textarea.html'})

In [5]:
#api.response_to_json(api.get('classesassuntosdisponiveis','?assunto=indenização'))
api.resources

{'users': 'http://0.0.0.0:8000/api/models/users/',
 'assuntos': 'http://0.0.0.0:8000/api/models/assuntos/',
 'classes': 'http://0.0.0.0:8000/api/models/classes/',
 'comarcas': 'http://0.0.0.0:8000/api/models/comarcas/',
 'competencias': 'http://0.0.0.0:8000/api/models/competencias/',
 'serventias': 'http://0.0.0.0:8000/api/models/serventias/',
 'classesassuntos': 'http://0.0.0.0:8000/api/models/classesassuntos/',
 'processos': 'http://0.0.0.0:8000/api/models/processos/',
 'tipospersonagem': 'http://0.0.0.0:8000/api/models/tipospersonagem/',
 'tiposmovimento': 'http://0.0.0.0:8000/api/models/tiposmovimento/',
 'tiposandamento': 'http://0.0.0.0:8000/api/models/tiposandamento/',
 'cargos': 'http://0.0.0.0:8000/api/models/cargos/',
 'funcionarios': 'http://0.0.0.0:8000/api/models/funcionarios/',
 'tiposdecisaorecurso': 'http://0.0.0.0:8000/api/models/tiposdecisaorecurso/',
 'tiposatojuiz': 'http://0.0.0.0:8000/api/models/tiposatojuiz/',
 'atosjuiz': 'http://0.0.0.0:8000/api/models/atosjuiz

In [12]:
api.response_to_json(api.get('personagensdisponiveis','?personagem=2na info'))

[{'id': '478314', 'nome_personagem': '2NA INFORMATICA E TERINAMENTO S/C LTDA'},
 {'id': '539763', 'nome_personagem': '2NA INFORMÁTICA E TREINAMENTO'},
 {'id': '532838',
  'nome_personagem': '2NA INFORMATICA E TREINAMENTO LTDA (SOS COMPUTADORES)'},
 {'id': '669413', 'nome_personagem': '2NA INFORMATICA E TREINAMENTO S/C LTDA'},
 {'id': '352640', 'nome_personagem': '2NA INFORMÁTICA TREINAMENTO'},
 {'id': '531177', 'nome_personagem': '2NA INFORMÁTICA TREINAMENTO'},
 {'id': '561687', 'nome_personagem': '2NA INFORMÁTICA TREINAMENTO'},
 {'id': '651623', 'nome_personagem': '2NA INFORMÁTICA TREINAMENTO'},
 {'id': '347667', 'nome_personagem': '2NA INFORMÁTICA TREINAMENTO S/C LTDA'},
 {'id': '364788', 'nome_personagem': '2NA INFORMÁTICA TREINAMENTO S/C LTDA'},
 {'id': '381517', 'nome_personagem': '2NA INFORMÁTICA TREINAMENTO S/C LTDA'},
 {'id': '381575', 'nome_personagem': '2NA INFORMÁTICA TREINAMENTO S/C LTDA'},
 {'id': '381611', 'nome_personagem': '2NA INFORMÁTICA TREINAMENTO S/C LTDA'},
 {'id'

In [38]:
api.resources

{'users': 'http://0.0.0.0:8000/api/models/users/',
 'assuntos': 'http://0.0.0.0:8000/api/models/assuntos/',
 'classes': 'http://0.0.0.0:8000/api/models/classes/',
 'comarcas': 'http://0.0.0.0:8000/api/models/comarcas/',
 'competencias': 'http://0.0.0.0:8000/api/models/competencias/',
 'serventias': 'http://0.0.0.0:8000/api/models/serventias/',
 'classesassuntos': 'http://0.0.0.0:8000/api/models/classesassuntos/',
 'processos': 'http://0.0.0.0:8000/api/models/processos/',
 'tipospersonagem': 'http://0.0.0.0:8000/api/models/tipospersonagem/',
 'tiposmovimento': 'http://0.0.0.0:8000/api/models/tiposmovimento/',
 'tiposandamento': 'http://0.0.0.0:8000/api/models/tiposandamento/',
 'cargos': 'http://0.0.0.0:8000/api/models/cargos/',
 'funcionarios': 'http://0.0.0.0:8000/api/models/funcionarios/',
 'tiposdecisaorecurso': 'http://0.0.0.0:8000/api/models/tiposdecisaorecurso/',
 'tiposatojuiz': 'http://0.0.0.0:8000/api/models/tiposatojuiz/',
 'atosjuiz': 'http://0.0.0.0:8000/api/models/atosjuiz

In [32]:
from django.db.models import Q

In [23]:
a = n2na[0]

In [74]:
api.resources

{'users': 'http://0.0.0.0:8000/api/models/users/',
 'assuntos': 'http://0.0.0.0:8000/api/models/assuntos/',
 'classes': 'http://0.0.0.0:8000/api/models/classes/',
 'comarcas': 'http://0.0.0.0:8000/api/models/comarcas/',
 'competencias': 'http://0.0.0.0:8000/api/models/competencias/',
 'serventias': 'http://0.0.0.0:8000/api/models/serventias/',
 'classesassuntos': 'http://0.0.0.0:8000/api/models/classesassuntos/',
 'processos': 'http://0.0.0.0:8000/api/models/processos/',
 'tipospersonagem': 'http://0.0.0.0:8000/api/models/tipospersonagem/',
 'tiposmovimento': 'http://0.0.0.0:8000/api/models/tiposmovimento/',
 'tiposandamento': 'http://0.0.0.0:8000/api/models/tiposandamento/',
 'cargos': 'http://0.0.0.0:8000/api/models/cargos/',
 'funcionarios': 'http://0.0.0.0:8000/api/models/funcionarios/',
 'tiposdecisaorecurso': 'http://0.0.0.0:8000/api/models/tiposdecisaorecurso/',
 'tiposatojuiz': 'http://0.0.0.0:8000/api/models/tiposatojuiz/',
 'atosjuiz': 'http://0.0.0.0:8000/api/models/atosjuiz

In [5]:
api.response_to_json(api.get('processossimilaresreport','?advogado=LUCIANO DA SILVA CORRÊA'))


{'count': 3844,
 'next': 'http://0.0.0.0:8000/api/models/processossimilaresreport/?advogado=LUCIANO+DA+SILVA+CORR%C3%8AA&page=2',
 'previous': None,
 'results': [{'id': 'http://0.0.0.0:8000/api/models/processossimilaresreport/1562077/',
   'similaridade': 98.0,
   'processo_base_tj': '2016.205.056459-8',
   'processo_base_cnj': '0056437-67.2016.8.19.0205',
   'processo_similar_tj': '2016.204.038735-7',
   'processo_similar_cnj': '0039489-53.2016.8.19.0204',
   'processo_base_comarca': 'Regional de Campo Grande',
   'processo_similar_comarca': 'Regional de Bangu',
   'processo_base_serventia': 'Cartório do 26º Juizado Especial Cível',
   'processo_similar_serventia': 'Cartório do 17º Juizado Especial Cível',
   'processo_base_assunto': 'Antecipação de Tutela / Tutela Específica',
   'processo_similar_assunto': 'Protesto Indevido de Título',
   'processo_base_classe': 'Procedimento do Juizado Especial Cível',
   'processo_similar_classe': 'Procedimento do Juizado Especial Cível',
   'pro

In [52]:
tj.Comarca.objects.filter(desc_coma__in=['Regional de Bangu'])

<QuerySet [<Comarca: Regional de Bangu>]>

In [156]:
personagem_processos = list(map(lambda p: p.processo_id,
                                            tj.PersonagemProcesso.objects.filter(
                                                Q(personagem__nome__icontains='DERVAL BARROS DE OLIVEIRA')|
                                                Q(personagem__nome='DERVAL BARROS DE OLIVEIRA')
                                            )))

In [157]:
personagem_processos

[]

In [160]:
tj.PersonagemProcesso.objects.filter(Q(personagem__nome='DERVAL BARROS DE OLIVEIRA'))

<QuerySet []>